In [3]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import keras.backend as K
from tensorflow import keras
from keras.layers import Reshape, Conv2DTranspose, PReLU
from keras.utils import to_categorical, plot_model
from keras.layers import Concatenate
import numpy as np
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.image import resize
from scipy.linalg import sqrtm
import tensorflow as tf
from IPython.display import clear_output, HTML
import glob
from keras.layers import AveragePooling2D, ZeroPadding2D, BatchNormalization, Activation, MaxPool2D, Add
from keras.layers import Normalization, Dense, Conv2D, Dropout, BatchNormalization, ReLU
from keras.models import Sequential, Model
from keras import Input
from keras.optimizers import *
from keras.callbacks import EarlyStopping
from keras.initializers import RandomNormal
from keras.layers import LeakyReLU, GlobalMaxPooling2D, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
import numpy as np
from skimage.transform import AffineTransform, warp
from keras import Sequential, Model
from keras.layers import Dense, Conv2D, Reshape, UpSampling2D, \
    BatchNormalization, Activation, Input, LeakyReLU, ZeroPadding2D, Dropout, Flatten, Conv2DTranspose
from keras.optimizers import Adam
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.ndimage import rotate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, Reshape, UpSampling2D, Conv2D, BatchNormalization, Activation, Input, LeakyReLU
from keras.initializers import RandomNormal
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.datasets.fashion_mnist import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
#import necessary libraries
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import cifar10
from keras.preprocessing import image
import keras.backend as K
import matplotlib.pyplot as plt
import numpy as np
import time
# import cv2
# from tqdm import tqdm
import numpy as np
import os
import matplotlib.pyplot as plt
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.datasets.fashion_mnist import load_data
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input, Dense, Reshape, Flatten, Conv2D, Conv2DTranspose
from keras.layers import LeakyReLU, Dropout, Embedding, Concatenate


In [4]:
# List physical GPUs and set memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [5]:
# Read the CSV file into a pandas DataFrame
df = pd.read_csv('emnist-letters-train.csv', delimiter=',', header=None)

In [6]:
df = df[df[0] != -1]

In [7]:
# Define the mapping dictionary
mapping = {1: 97, 2: 98, 3: 99, 4: 100, 5: 101, 6: 102, 7: 103, 8: 104, 9: 105, 
           10: 106, 11: 107, 12: 108, 13: 109, 14: 110, 15: 111, 16: 112, 17: 113, 
           18: 114, 19: 115, 20: 116, 21: 117, 22: 118, 23: 119, 24: 120, 25: 121, 
           26: 122, 27: 123}

        # Map the labels column to its corresponding value
df[0] = df[0].map(mapping)

In [9]:
y_pre = np.array(df.iloc[:,0].values)
y_pre = pd.Categorical(y_pre)
X = np.array(df.iloc[:,1:].values)
X = X.reshape(-1,28,28,1)
preprocessed = []
for image in X:
    rotated_image = rotate(image, 90, reshape=False)
    flipped_image = np.flipud(rotated_image)
    preprocessed.append(flipped_image)
X_pre = np.array(preprocessed)
X_train,X_test,y_train,y_test = train_test_split(X_pre,y_pre,test_size=0.1,random_state=42)

In [ ]:
def train(self, images, labels_input, epochs=10000, batch_size=256, save_interval=500, gen_steps=1):
        os.makedirs('ACGAN_Generated_Images', exist_ok=True)
        X_train = images
        y_train = labels_input

        batches_per_epoch = X_train.shape[0] // batch_size

        for epoch in range(epochs):
            for batch in range(batches_per_epoch):
                # Train Discriminator
                idx = np.random.randint(0, X_train.shape[0], batch_size)
                imgs = X_train[idx]
                labels_real = np.ones((batch_size, 1))  # Real labels

                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
                gen_labels = np.random.randint(0, self.num_classes, (batch_size, 1))
                gen_imgs = self.generator.predict([noise, gen_labels])
                labels_fake = np.zeros((batch_size, 1))  # Fake labels

                d_loss_real = self.discriminator.train_on_batch(imgs, [labels_real, y_train[idx]])
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, [labels_fake, gen_labels])
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # Train Generator
                g_loss = None
                for _ in range(gen_steps):
                    noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
                    gen_labels = np.random.randint(0, self.num_classes, (batch_size, 1))
                    valid_y = np.ones((batch_size, 1))
                    g_loss = self.acgan.train_on_batch([noise, gen_labels], [valid_y, gen_labels])

                # Print the progress
                print(f"Epoch {epoch + 1}/{epochs}, Batch {batch + 1}/{batches_per_epoch} [D loss: {d_loss[0]}, acc.: {100 * d_loss[3]:.2f}%] [G loss: {g_loss[0]}]")

            # Save generated images at save intervals
            if (epoch + 1) % save_interval == 0:
                self.save_images(epoch+1)

In [ ]:
class CGAN():

    def __init__(self, rows, cols, channels, z = 100):
        # Input shape
        self.img_rows = rows
        self.img_cols = cols
        self.channels = channels
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = z
        self.classes = 25
        optimizer = Adam(0.00002, 0.5)
        # Build and compile the discriminator
        self.discriminator = self.define_discriminator(self.img_shape,self.classes)
        self.discriminator.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])
        # Build the generator
        self.g_model = self.define_generator(self.latent_dim,self.classes)
        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.g_model(z)
        # For the combined model we will only train the generator
        self.discriminator.trainable = False
        # The discriminator takes generated images as input and
        # determines validity
        valid = self.discriminator(img)
        # The combined model (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy',optimizer=optimizer)

    # define the standalone discriminator model
    def define_discriminator(self, in_shape, n_classes=25):
        in_label = Input(shape=(1,))
        li = Embedding(n_classes, 50)(in_label)
        n_nodes = in_shape[0] * in_shape[1]
        li = Dense(n_nodes)(li)
        li = Reshape((in_shape[0], in_shape[1], 1))(li)
        in_image = Input(shape=in_shape)
        merge = Concatenate()([in_image, li])
        fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(merge)
        fe = LeakyReLU(alpha=0.2)(fe)
        fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(fe)
        fe = LeakyReLU(alpha=0.2)(fe)
        fe = Flatten()(fe)
        fe = Dropout(0.4)(fe)
        out_layer = Dense(1, activation='sigmoid')(fe)
        model = Model([in_image, in_label], out_layer)
        return model

    # define the standalone generator model
    def define_generator(self, latent_dim, n_classes):
        in_label = Input(shape=(1,))
        li = Embedding(n_classes, 50)(in_label)
        n_nodes = 7 * 7
        li = Dense(n_nodes)(li)
        li = Reshape((7, 7, 1))(li)
        in_lat = Input(shape=(latent_dim,))
        n_nodes = 128 * 7 * 7
        gen = Dense(n_nodes)(in_lat)
        gen = LeakyReLU(alpha=0.2)(gen)
        gen = Reshape((7, 7, 128))(gen)
        merge = Concatenate()([gen, li])
        gen = Conv2DTranspose(64, (4,4), strides=(2,2), padding='same')(merge)
        gen = LeakyReLU(alpha=0.2)(gen)
        gen = Conv2DTranspose(64, (4,4), strides=(2,2), padding='same')(gen)
        gen = LeakyReLU(alpha=0.2)(gen)
        out_layer = Conv2D(1, (7,7), activation='tanh', padding='same')(gen)
        model = Model([in_lat, in_label], out_layer)
        return model

    # define the combined generator and discriminator model, for updating the generator
    def define_gan(self, g_model, d_model):
        d_model.trainable = False
        gen_noise, gen_label = g_model.input
        gen_output = g_model.output
        gan_output = d_model([gen_output, gen_label])
        model = Model([gen_noise, gen_label], gan_output)
        opt = Adam(lr=0.0002, beta_1=0.5)
        model.compile(loss='binary_crossentropy', optimizer=opt)
        return model

    def load_real_samples(self):
        trainX, trainy = X_pre, y_pre
        X = expand_dims(trainX, axis=-1)
        X = X.astype('float32')
        X = (X - 127.5) / 127.5
        return [X, trainy]

    # select real samples
    def generate_real_samples(self, dataset, n_samples):
        images, labels = dataset
        ix = randint(0, images.shape[0], n_samples)
        X, labels = images[ix], labels[ix]
        y = ones((n_samples, 1))
        return [X, labels], y

    # generate points in latent space as input for the generator
    def generate_latent_points(self, latent_dim, n_samples, n_classes=26):
        x_input = randn(latent_dim * n_samples)
        z_input = x_input.reshape(n_samples, latent_dim)
        labels = randint(0, n_classes, n_samples)
        return [z_input, labels]

    # use the generator to generate n fake examples, with class labels
    def generate_fake_samples(self, generator, latent_dim, n_samples):
        z_input, labels_input = generate_latent_points(latent_dim, n_samples)
        images = generator.predict([z_input, labels_input])
        y = zeros((n_samples, 1))
        return [images, labels_input], y

    # function to save generated images
    def save_imgs(self, generator, latent_dim, epoch):
        r, c = 5, 5
        z_input, labels_input = generate_latent_points(latent_dim, r * c)
        gen_imgs = generator.predict([z_input, labels_input])
        gen_imgs = 0.5 * gen_imgs + 0.5
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        os.makedirs('CGAN_Generated_Mnist', exist_ok=True)
        fig.savefig("CGAN_Generated_Mnist/CGAN_mnist_{:d}.png".format(epoch))
        plt.close()

    # train the generator and discriminator
    def train(self, g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=128):
        bat_per_epo = int(dataset[0].shape[0] / n_batch)
        half_batch = int(n_batch / 2)
        for epoch in range(n_epochs):
            [X_real, labels_real], y_real = generate_real_samples(dataset, half_batch)
            d_loss1, _ = d_model.train_on_batch([X_real, labels_real], y_real)
            [X_fake, labels], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            d_loss2, _ = d_model.train_on_batch([X_fake, labels], y_fake)
            [z_input, labels_input] = generate_latent_points(latent_dim, n_batch)
            y_gan = ones((n_batch, 1))
            g_loss = gan_model.train_on_batch([z_input, labels_input], y_gan)
            # print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' % (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
            print ("%d [D1 loss: %.3f:%.3f,  acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss1,d_loss2,100*d_loss1, g_loss))
        if n_epochs % 50 == 0:
            save_imgs(g_model, latent_dim, epoch)
        g_model.save('CGAN_generator.h5')

    # size of the latent space
    gan_model = define_gan(g_model, d_model)
    dataset = load_real_samples()
    train(g_model, d_model, gan_model, dataset, latent_dim)
